<a href="https://colab.research.google.com/github/lordoz234/ordoz234-Tabular-Playground-Series---Oct-2021/blob/main/Intelex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install autogluon.tabular[all]

In [7]:
!pip install scikit-learn-intelex

In [8]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from timeit import default_timer as timer
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [9]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [10]:
train_data = pd.read_csv("/content/drive/My Drive/X5-MIPT/tabular-playground-series-oct-2021/train.csv")
test_data = pd.read_csv("/content/drive/My Drive/X5-MIPT/tabular-playground-series-oct-2021/test.csv")
submission = pd.read_csv("/content/drive/My Drive/X5-MIPT/tabular-playground-series-oct-2021/sample_submission.csv")

In [11]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 287 entries, id to target
dtypes: float64(240), int64(47)
memory usage: 2.1 GB


In [12]:
features = [col for col in train_data.columns if 'f' in col]

cont_features =[]
disc_features =[]

for col in features:
    if train_data[col].dtype=='float64':
        cont_features.append(col)
    else:
        disc_features.append(col)
        
train_data[cont_features] = train_data[cont_features].astype('float32')
train_data[disc_features] = train_data[disc_features].astype('uint8')
train_data[cont_features] = train_data[cont_features].astype('float32')
train_data[disc_features] = train_data[disc_features].astype('uint8')

In [13]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 287 entries, id to target
dtypes: float32(240), int64(2), uint8(45)
memory usage: 973.7 MB


In [14]:
import logging
from sklearnex import patch_sklearn

logger = logging.getLogger()
logger.setLevel(10)

patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [15]:
!export SKLEARNEX_VERBOSE=INFO

In [16]:
from sklearn.model_selection import train_test_split

t0 = timer()
X_train, X_val = train_test_split(train_data, test_size = 0.2)
t1 = timer()

INFO:root:sklearn.model_selection.train_test_split: running accelerated version on CPU


In [17]:
print("Train_test_split time: {} seconds".format(t1 - t0))

Train_test_split time: 2.0403424680000626 seconds


In [18]:
from  autogluon.tabular import TabularPredictor

random_state = 55
label = 'target'

hyperparameters = {
    'GBM': [
        {'extra_trees': True, 'seed': random_state, 'ag_args': {'name_suffix': 'XT'}},
        {},
    ],
    'RF': [
        {'criterion': 'gini', 'random_state': random_state,
         'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary']},
         'ag_args_fit': {'use_daal': True}},
    ],
    'KNN': [
        {'weights': 'uniform', 'n_neighbors': 1000, 'ag_args': {'name_suffix': 'Unif'},
        'ag_args_fit': {'use_daal': True}},
        {'weights': 'distance', 'n_neighbors': 1000, 'ag_args': {'name_suffix': 'Dist'},
        'ag_args_fit': {'use_daal': True}},
    ],
}

t0 = timer()

autogluon_predictor = TabularPredictor(
    label = label,
    eval_metric = "roc_auc",
    learner_kwargs={'ignored_columns': ['id']}
).fit(
    train_data=X_train,
    verbosity = 4, 
    hyperparameters=hyperparameters
)

t1 = timer()
fitting_time = t1 - t0

Level 25:autogluon.core.utils.utils:No path specified. Models will be saved in: "AutogluonModels/ag-20211010_160817/"
INFO:root:============ fit kwarg info ============
INFO:root:User Specified kwargs:
INFO:root:{'verbosity': 4}
INFO:root:Full kwargs:
INFO:root:{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': False,
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'num_bag_folds': None,
 'num_bag_sets': None,
 'num_stack_levels': None,
 'quantile_levels': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 4}
INFO:root:========================================
INFO:root:Warning: Training may take a very long time because `time_limit` was not specified and `train_data` is large (800000 samples, 823.2 

[1]	train_set's binary_logloss: 0.678739	valid_set's binary_logloss: 0.678982
[2]	train_set's binary_logloss: 0.665607	valid_set's binary_logloss: 0.66598
[3]	train_set's binary_logloss: 0.653528	valid_set's binary_logloss: 0.654165
[4]	train_set's binary_logloss: 0.642415	valid_set's binary_logloss: 0.643234
[5]	train_set's binary_logloss: 0.632392	valid_set's binary_logloss: 0.633464
[6]	train_set's binary_logloss: 0.623071	valid_set's binary_logloss: 0.624243
[7]	train_set's binary_logloss: 0.614717	valid_set's binary_logloss: 0.616154
[8]	train_set's binary_logloss: 0.606894	valid_set's binary_logloss: 0.608457
[9]	train_set's binary_logloss: 0.599602	valid_set's binary_logloss: 0.601341
[10]	train_set's binary_logloss: 0.592809	valid_set's binary_logloss: 0.594663
[11]	train_set's binary_logloss: 0.58679	valid_set's binary_logloss: 0.588862
[12]	train_set's binary_logloss: 0.581033	valid_set's binary_logloss: 0.583364
[13]	train_set's binary_logloss: 0.576032	valid_set's binary_lo

INFO:root:sklearn.metrics.roc_auc_score: running accelerated version on CPU
Level 15:autogluon.core.utils.savers.save_pkl:Saving AutogluonModels/ag-20211010_160817/models/LightGBMXT/model.pkl
INFO:autogluon.tabular.trainer.abstract_trainer:	0.854	 = Validation score   (roc_auc)
INFO:autogluon.tabular.trainer.abstract_trainer:	1814.76s	 = Training   runtime
INFO:autogluon.tabular.trainer.abstract_trainer:	0.79s	 = Validation runtime
Level 15:autogluon.core.utils.savers.save_pkl:Saving AutogluonModels/ag-20211010_160817/models/trainer.pkl
INFO:autogluon.tabular.trainer.abstract_trainer:Fitting model: LightGBM ...
DEBUG:autogluon.core.models.abstract.abstract_model:	Dropped 0 of 285 features.
Level 15:autogluon.core.models.abstract.abstract_model:	Fitting LightGBM with 'num_gpus': 0, 'num_cpus': 2
Level 15:autogluon.tabular.models.lgb.lgb_model:Training Gradient Boosting Model for 10000 rounds...
Level 15:autogluon.tabular.models.lgb.lgb_model:with the following hyperparameter settings:
L

[1]	train_set's binary_logloss: 0.677507	valid_set's binary_logloss: 0.677809
[2]	train_set's binary_logloss: 0.663355	valid_set's binary_logloss: 0.663878
[3]	train_set's binary_logloss: 0.650541	valid_set's binary_logloss: 0.651311
[4]	train_set's binary_logloss: 0.638891	valid_set's binary_logloss: 0.639878
[5]	train_set's binary_logloss: 0.628288	valid_set's binary_logloss: 0.629498
[6]	train_set's binary_logloss: 0.618613	valid_set's binary_logloss: 0.620013
[7]	train_set's binary_logloss: 0.609781	valid_set's binary_logloss: 0.611396
[8]	train_set's binary_logloss: 0.601697	valid_set's binary_logloss: 0.603459
[9]	train_set's binary_logloss: 0.594302	valid_set's binary_logloss: 0.596298
[10]	train_set's binary_logloss: 0.587499	valid_set's binary_logloss: 0.589655
[11]	train_set's binary_logloss: 0.581267	valid_set's binary_logloss: 0.583644
[12]	train_set's binary_logloss: 0.575519	valid_set's binary_logloss: 0.578081
[13]	train_set's binary_logloss: 0.570258	valid_set's binary_

INFO:root:sklearn.metrics.roc_auc_score: running accelerated version on CPU
Level 15:autogluon.core.utils.savers.save_pkl:Saving AutogluonModels/ag-20211010_160817/models/LightGBM/model.pkl
INFO:autogluon.tabular.trainer.abstract_trainer:	0.8534	 = Validation score   (roc_auc)
INFO:autogluon.tabular.trainer.abstract_trainer:	1680.49s	 = Training   runtime
INFO:autogluon.tabular.trainer.abstract_trainer:	0.67s	 = Validation runtime
Level 15:autogluon.core.utils.savers.save_pkl:Saving AutogluonModels/ag-20211010_160817/models/trainer.pkl
INFO:autogluon.tabular.trainer.abstract_trainer:Fitting model: RandomForestGini ...
DEBUG:autogluon.core.models.abstract.abstract_model:	Dropped 0 of 285 features.
Level 15:autogluon.core.models.abstract.abstract_model:	Fitting RandomForestGini with 'num_gpus': 0, 'num_cpus': 2
Level 15:autogluon.tabular.models.rf.rf_model:	Using daal4py RF backend...
INFO:root:sklearn.ensemble.RandomForestClassifier.fit: running accelerated version on CPU
INFO:root:skle

In [19]:
print("Fitting time: {} seconds".format(fitting_time))

Fitting time: 5592.453407424 seconds


In [20]:
t0 = timer()
leaderboard = autogluon_predictor.leaderboard(X_val)
t1 = timer()

Level 15:autogluon.core.utils.loaders.load_pkl:Loading: AutogluonModels/ag-20211010_160817/models/LightGBM/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: AutogluonModels/ag-20211010_160817/models/LightGBMXT/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: AutogluonModels/ag-20211010_160817/models/RandomForestGini/model.pkl
INFO:root:sklearn.ensemble.RandomForestClassifier.predict_proba: running accelerated version on CPU
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: AutogluonModels/ag-20211010_160817/models/WeightedEnsemble_L2/model.pkl
INFO:root:sklearn.metrics.roc_auc_score: running accelerated version on CPU
INFO:root:sklearn.metrics.roc_auc_score: running accelerated version on CPU
INFO:root:sklearn.metrics.roc_auc_score: running accelerated version on CPU
INFO:root:sklearn.metrics.roc_auc_score: running accelerated version on CPU
DEBUG:autogluon.tabular.learner.abstract_learner:Model scores:
DEBUG:autogluon.tabular.learner.abstract_lea

                 model  score_test  score_val  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2    0.857033   0.854196       34.624098       1.464113  3497.632920                 0.331521                0.007615           2.381338            2       True          4
1           LightGBMXT    0.856641   0.853969       17.957506       0.791431  1814.760746                17.957506                0.791431        1814.760746            1       True          1
2             LightGBM    0.856449   0.853377       16.335071       0.665066  1680.490836                16.335071                0.665066        1680.490836            1       True          2
3     RandomForestGini    0.831900   0.828599       38.759614       0.391794  1968.545976                38.759614                0.391794        1968.545976            1       True          3


In [21]:
print("Getting leaderboard time: {} seconds".format(t1 - t0))

Getting leaderboard time: 74.30426075299965 seconds


In [22]:
t0 = timer()
predictions = autogluon_predictor.predict_proba(test_data)
t1 = timer()
prediction_time = t1 - t0

Level 15:autogluon.core.utils.loaders.load_pkl:Loading: AutogluonModels/ag-20211010_160817/models/WeightedEnsemble_L2/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: AutogluonModels/ag-20211010_160817/models/LightGBM/model.pkl
Level 15:autogluon.core.utils.loaders.load_pkl:Loading: AutogluonModels/ag-20211010_160817/models/LightGBMXT/model.pkl


In [23]:
print("Predictions time: {} seconds".format(prediction_time))

Predictions time: 89.61354125199978 seconds
